In [1]:
%run init_notebook.py

In [2]:
import eurostat
from src.utils import load_pd_df, get_dt_index, save_pd_df
from pandas_datareader.fred import FredReader

In [3]:
# interest rates

In [11]:
depo = FredReader('ECBDFR', start=datetime(2019,1,1), end=datetime.today()).read()
depo = depo.rename(columns={'ECBDFR': 'ecb_rate'}) / 100

In [12]:
df = pd.read_csv(os.path.join(DATA_DIR, 'ecb_interest_rates.csv'), low_memory=False)
df = df[df['REF_AREA'] == 'DE'].dropna(axis=1)
df['date'] = df['TIME_PERIOD'].apply(lambda x: datetime.strptime(x, "%Y-%m"))
df = df[df['TITLE'] == list(set(df['TITLE']))[1]]
df.set_index('date', inplace=True)

mir = df[['OBS_VALUE']].rename(columns={'OBS_VALUE': 'mir'}) / 100

In [13]:
mir = mir.join(depo)

In [14]:
mir = mir.ffill()

In [17]:
save_pd_df(mir, 'mir.csv')

In [10]:
# eurostat

In [11]:
codes = eurostat.get_toc_df()

In [12]:
[i for i in codes['title'] if 'deposit'.lower() in i.lower()]

['Loan and deposit one year interest rate']

In [13]:
filt = [
    'HICP - energy', 
    'HICP - food, alcohol and tobacco',
    'Gross domestic product (GDP) at current market prices by metropolitan regions',
    'Gross domestic product (GDP) at market prices - quarterly data',
    'HICP - inflation rate',
    'HICP - all items excluding energy and seasonal food',
    'HICP - energy',
    'HICP - monthly data (index)',
    'Final consumption expenditure',
    'Unemployment rate',
    'Unemployment rate by education level',
    'Unemployment rate (%) - monthly data',
    'Loan and deposit one year interest rate',
]
filt = [i in filt for i in codes['title']]

In [14]:
codes[filt]

,title,code,type,last update of data,last table structure change,data start,data end
132,Final consumption expenditure,MED_EC2,dataset,2020-05-28T23:00:00+0200,2020-05-28T23:00:00+0200,2005,2018
192,Unemployment rate,MED_PS421,dataset,2020-05-28T23:00:00+0200,2020-05-28T23:00:00+0200,2005,2019
193,Unemployment rate by education level,MED_PS422,dataset,2016-08-29T23:00:00+0200,2023-12-21T23:00:00+0100,2000,2015
214,Gross domestic product (GDP) at current market...,MET_10R_3GDP,dataset,2024-02-28T23:00:00+0100,2024-02-28T23:00:00+0100,2000,2022
1327,HICP - monthly data (index),PRC_HICP_MIDX,dataset,2024-03-01T11:00:00+0100,2024-03-01T11:00:00+0100,1996-01,2024-02
1330,HICP - monthly data (index),PRC_HICP_MIDX$DV_1082,dataset,2024-03-01T11:00:00+0100,2024-03-01T11:00:00+0100,None,None
1333,HICP - monthly data (index),PRC_HICP_MIDX$DV_651,dataset,2024-03-01T11:00:00+0100,2024-03-01T11:00:00+0100,None,None
2148,Unemployment rate (%) - monthly data,EI_LMHR_M,dataset,2024-03-01T16:21:00+0100,2024-03-01T16:21:00+0100,1983-01,2024-01
2234,Loan and deposit one year interest rate,ENPE_IRT_LD,dataset,2024-01-31T23:00:00+0100,2023-11-28T11:00:00+0100,2005,2022
5160,Gross domestic product (GDP) at market prices ...,TIPSAU20,dataset,2024-03-05T23:00:00+0100,2024-03-05T23:00:00+0100,1995-Q1,2023-Q4


In [10]:
# overall inflation

In [11]:
data = eurostat.get_data('med_ec2') #, filter_pars={'geo': ['DE','FR']})

In [11]:
df = pd.DataFrame(data[1:], columns=data[0]).transpose()
df = df.loc[:, (df.loc['unit'] == 'I05') & (df.loc['coicop'] == 'CP00')]
df.columns = [f'cpi_{i}' for i in df.loc['geo\TIME_PERIOD'].values]

cpi = df.iloc[5:].copy().dropna().astype(float)
cpi.index = [datetime(int(i[:4]), int(i[5:7]), 1) for i in cpi.index]

In [12]:
cpi['pi_de'] = np.log(cpi['cpi_DE']).diff()

In [13]:
save_pd_df(cpi, 'cpi.csv')

In [14]:
# food tobaco inflation

In [15]:
data = eurostat.get_data('TEICP260', filter_pars={'geo': ['DE']})
data = np.array(data)

In [16]:
df = pd.DataFrame(data=data[1:], columns=data[0])
df

,freq,coicop,unit,geo\TIME_PERIOD,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01
0,M,FOOD,I15,DE,139.5,141.3,141.1,141.1,141.2,141.2,141.0,141.5,141.6,142.4,142.4,None
1,M,FOOD,PCH_M1,DE,2.0,1.3,-0.1,0.0,0.1,0.0,-0.1,0.4,0.1,0.6,0.0,0.9
2,M,FOOD,PCH_M12,DE,18.0,18.6,15.5,13.6,12.6,10.7,9.2,8.1,7.1,6.6,5.8,5.1


In [3]:
# pi prognosis

In [4]:
df = get_dt_index(load_pd_df('ecb_prognosis_hicp_dec_2021.csv'), dt_index_col="DATE").iloc[:, 1:]
df.index.name = 'date'
df.columns = ["infl_forecast"]

In [5]:
df1 = load_pd_df('ecb_prognosis_hicp_jun_2021.csv')
df1['date'] = df1.DATE.apply(lambda x: datetime(int(x[:4]), 6, 30))
df1 = df1.drop('DATE', axis=1).set_index('date').iloc[:, 1:]
df1.columns = ["infl_forecast"]

df = pd.concat([df, df1], axis=0).sort_index()

In [19]:
save_pd_df(df, 'infl_prognonsis.csv')

# hhp

In [3]:
dfs, metadata = [], []
for i, file in enumerate(os.listdir(os.path.join(DATA_DIR, 'hhs'))):
    print(file)
    try:
        dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file), )
        dfs.append(dta)
        metadata.append(metadata)
    except Exception as e:
        print(e)
        continue

bophh_suf_202304_v01_wave01.dta
bophh_suf_202304_v01_wave02.dta
bophh_suf_202304_v01_wave03.dta
bophh_suf_202304_v01_wave04.dta


C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_21236\2310966925.py:5: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file), )


bophh_suf_202304_v01_wave05.dta
bophh_suf_202304_v01_wave06.dta
bophh_suf_202304_v01_wave07.dta
bophh_suf_202304_v01_wave08.dta
bophh_suf_202304_v01_wave09.dta
bophh_suf_202304_v01_wave10.dta
bophh_suf_202304_v01_wave11.dta
bophh_suf_202304_v01_wave12.dta
bophh_suf_202304_v01_wave13.dta
bophh_suf_202304_v01_wave14.dta
bophh_suf_202304_v01_wave15.dta
bophh_suf_202304_v01_wave16.dta
bophh_suf_202304_v01_wave17.dta
bophh_suf_202304_v01_wave18.dta
bophh_suf_202304_v01_wave19.dta
bophh_suf_202304_v01_wave20.dta
bophh_suf_202304_v01_wave21.dta
bophh_suf_202304_v01_wave22.dta
bophh_suf_202304_v01_wave23.dta
bophh_suf_202304_v01_wave24.dta
bophh_suf_202304_v01_wave25.dta
bophh_suf_202304_v01_wave26.dta
bophh_suf_202304_v01_wave27.dta
'utf-8' codec can't decode byte 0xfc in position 8: invalid start byte
bophh_suf_202304_v01_wave28.dta
'utf-8' codec can't decode byte 0x80 in position 5: invalid start byte
bophh_suf_202304_v01_wave29.dta
bophh_suf_202304_v01_wave30.dta


C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_21236\2310966925.py:5: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file), )


bophh_suf_202304_v01_wave31.dta


...

bophh_suf_202304_v01_wave39.dta


C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_21236\2310966925.py:5: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file), )


bophh_suf_202304_v01_wave40.dta


C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_21236\2310966925.py:5: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file), )


bophh_suf_202304_v01_wave41.dta
bophh_suf_202304_v01_wave42.dta
bophh_suf_202304_v01_wave43.dta
bophh_suf_202304_v01_wave44.dta
bophh_suf_202304_v01_wave45.dta


In [76]:
data = pd.concat(dfs, axis=0)

In [77]:
data['date_recorded'] = data[['year', 'month', 'day']].apply(lambda x: datetime(*x.values.astype(int)), axis=1).rename('date_recorded')

C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_21236\422614078.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['date_recorded'] = data[['year', 'month', 'day']].apply(lambda x: datetime(*x.values.astype(int)), axis=1).rename('date_recorded')


In [78]:
dict_cols_rename = {'inflexppoint': 'pi_exp', 'inflexppoint_long': 'pi_exp_long', 'expint_sav': 'i_exp', 'devinfpoint': 'pi_perc'}

In [84]:
dict_cols_parse = {
    'id': float,
    # 'riskaversion': str,
    # 'date_recorded': 
    'i_exp': float,
    'pi_perc': float,
    'hhinc': str,
    'pinc': str,
    'pi_exp': float,
    'eduschool': str,
 'eduwork': str,
 'employ': str,
 'profession': str,
 'hhsize': str,
 'hhchildren': str,
 'familystatus': str,
 'hhinc': str,
 'pinc': str,
    'homeown': str,
    'pi_exp_long': float,
}
dict_cols_parse.update({f'infexprob_{i}': str for i in list(map(chr, range(97, 107)))})
dict_cols_parse.update({f'incexp_{i}': str for i in list(map(chr, range(97, 107)))})
dict_cols_parse.update({f'mainshopper_{i}': str for i in list(map(chr, range(97, 101)))})
dict_cols_parse.update({f'netwealth_detail_{i}': str for i in list(map(chr, range(97, 104)))})
# dict_cols_parse.update({f'mainshopper_{i}': str for i in list(map(chr, range(97, 1007)))})

lst_cols = list(dict_cols_parse.keys())
lst_cols.extend(['date_recorded'])

_ = [
        "don't know",
        'no answer',
        'does not apply',
        'nan',
    ]
dict_parse_nan = dict(zip(_, list([np.nan]) * len(_)))


In [85]:
dict_inc_replace = {'3.500 to 3.999 EUR': '3.500 to 3.999 EUR',
 '2.000 to 2.499 EUR': '2.000 to 2.499 EUR',
 '500 to 999 EUR': '500 to 999 EUR',
 '3000 - < 3500 EUR': '3.000 to 3.499 EUR',
 '3500 - < 4000 EUR': '3.500 to 3.999 EUR',
 'under 500 EUR': 'under 500 EUR',
 '> 4499 EUR': '5.000 to 5.999 EUR',
 '1000 - < 1500 EUR': '1.000 to 1.499 EUR',
 '2000 - < 2500 EUR': '2.000 to 2.499 EUR',
 np.nan: np.nan,
 '< 500 EUR': 'under 500 EUR',
 '3.000 to 3.499 EUR': '3.000 to 3.499 EUR',
 '6.000 to 7.999 EUR': '6.000 to 7.999 EUR',
 '4000 - < 4500 EUR': '4.000 to 4.999 EUR',
 '4.000 to 4.999 EUR': '4.000 to 4.999 EUR',
 '5.000 to 5.999 EUR': '5.000 to 5.999 EUR',
 '10.000 EUR and more': '10.000 EUR and more',
 '6.000 EUR and more': '6.000 to 7.999 EUR',
 '2500 - < 3000 EUR': '2.500 to 2.999 EUR',
 '8.000 to 9.999 EUR': '8.000 to 9.999 EUR',
 '1500 - < 2000 EUR': '1.500 to 1.999 EUR',
 '1.500 to 1.999 EUR': '1.500 to 1.999 EUR',
 '500 - < 1000 EUR': '500 to 999 EUR',
  '500 bis 999 EUR': '500 to 999 EUR',
 '2.500 to 2.999 EUR': '2.500 to 2.999 EUR',
 '1.000 to 1.499 EUR': '1.000 to 1.499 EUR'}
dict_inc_replace.update(dict_parse_nan)

In [86]:
data = data.rename(columns=dict_cols_rename)

In [87]:
# df

In [88]:
df = data[lst_cols]

In [89]:
df = df.reset_index(drop=True)

In [90]:
# del data, dfs

In [91]:
df['date_forecast'] = df.date_recorded + timedelta(days=365)

In [110]:
df['hhinc'] = df.hhinc.map(dict_inc_replace)
df['pinc'] = df['pinc'].map(dict_inc_replace)
for col in ['pi_exp', 'pi_perc', 'i_exp', 'pi_exp_long',]:
    df = df.drop(col, axis=1).join(df[col].apply(lambda x: dict_parse_nan[x] if x in dict_parse_nan.keys() else x))

In [93]:
cols = [i for i in dict_cols_parse if sum([term in i for term in ['infexprob', 'incexp']]) > 0]
for col in cols:
    df[col] = df[col].replace(dict_parse_nan)

C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_21236\3703843038.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(dict_parse_nan)


In [94]:
# midpoints

In [95]:
dict_hhinc_midpoint = {'1.000 to 1.499 EUR': 1250,
 '1.500 to 1.999 EUR': 1750,
 '10.000 EUR and more': 15000,
 '2.000 to 2.499 EUR': 2250,
 '2.500 to 2.999 EUR': 2750,
 '3.000 to 3.499 EUR': 3250,
 '3.500 to 3.999 EUR': 3750,
 '4.000 to 4.999 EUR': 4500,
 '5.000 to 5.999 EUR': 5500,
 '500 to 999 EUR': 750,
 '6.000 to 7.999 EUR': 7000,
 '8.000 to 9.999 EUR': 9000,
 None: np.nan,
 'under 500 EUR': 250}

In [96]:
df['pinc_midpoint'] = df['pinc'].map(dict_hhinc_midpoint)
df = df.sort_values('date_recorded')
sub = df.set_index(['id', 'date_recorded',])['pinc_midpoint'].unstack()
sub = sub.ffill(axis=1).diff(axis=1).stack().rename('pinc_delta')
df = df.set_index(['id', 'date_recorded']).join(sub).reset_index()

In [97]:
df['hhinc_midpoint'] = df['hhinc'].map(dict_hhinc_midpoint)
df = df.sort_values('date_recorded')
sub = df.set_index(['id', 'date_recorded',])['hhinc_midpoint'].unstack()
sub = sub.ffill(axis=1).diff(axis=1).stack().rename('hhinc_delta')
df = df.set_index(['id', 'date_recorded']).join(sub).reset_index()

In [98]:
dict_educ_work = {
    ' Completed vocational training (vocational school or commercial college)': 'vocational',
 ' Currently in training or studying (bachelor’s degree not yet completed)': 'student',
 'Another professional qualification': 'vocational',
 'Bachelor’s degree, applied sciences degree, completed training at an engineering college': 'bachelor',
 'Completed training at a technical or commercial college, school for master craftsmen or engineers or university of co-operative education with shorter preparation time (up to 880 hours)': 'meister',
 'Completed training at a university of cooperative education with longer preparation time (more than 880 hours)': 'bachelor',
 'Completed vocational training (apprenticeship)': 'vocational',
 'Diploma or master’s degree, completed teacher training course': 'masters',
 'Doctorate / postdoctoral qualification obtained': 'phd',
 'No vocational training or university degree  completed': 'high school',
 'apprenticeship': 'vocational',
 'doctorate': 'phd',
 'in training/studying': 'student',
 'master/diploma': 'masters',
 'no degree': 'high school',
 'technical or commercial college': 'bachelor',
 'university of cooperative education': 'bachelor',
 'vocational school': 'vocational',
}
dict_educ_work.update(dict_parse_nan)

In [99]:
df['eduwork_slim'] = df['eduwork'].apply(lambda x: dict_educ_work[x] if x in dict_educ_work.keys() else x).dropna()

In [100]:
dict_employ = {'Apprenticeship': 'in educ',
 'Low-paid part-time (Mini-Job)': 'Infrequent employment',
 'Military service / BuFDi / FSJ': 'in educ',
 'One-euro job': 'Low-paid part-time (Mini-Job)',
 'Partial retirement scheme': 'Infrequent employment',
 'Retraining': 'voluntary year',
 'casual or irregular employment': 'Infrequent employment',
 'early retirement': 'retirement',
 'full-time employment': 'Employed, full-time',
 'other non-employment': 'Unemployed',
 'parental leave': 'voluntary year',
 'part-time employment': 'Employed, part-time',
 'student/internship': 'in educ',
 'unemployed': 'Unemployed',
              }
dict_employ.update(dict_parse_nan)

In [101]:
df['employ_slim'] = df['employ'].apply(lambda x: dict_employ[x] if x in dict_employ.keys() else x)

In [102]:
# parse nan
for col in ['profession', 'hhsize', *[i for i in df.columns if 'mainshopper' in i], *[i for i in df.columns if 'netwealth' in i]]:
    df[col] = df[col].apply(lambda x: dict_parse_nan[x.lower()] if x in dict_parse_nan.keys() else x)

In [103]:
# a-d cats

In [104]:
df['is_food_shop'] = ~(df['mainshopper_a'] == 'rather my partner')
df['is_invest_shop'] = ~(df['mainshopper_b'] == 'rather my partner')
df['is_invest_manag'] = ~(df['mainshopper_d'] == 'rather my partner')

In [105]:
dict_networth_mid_point = {'1 to 2.500 EUR': 1250,
 '10.000 to 25.000 EUR': 17500,
 '100.000 to 250.000 EUR': 175000,
 '2.500 to 5.000 EUR': 3250,
 '25.000 to 50.000 EUR': 37500,
 '250.000 to 500.000 EUR': 750000,
 '5.000 to 10.000 EUR': 7500,
 '50.000 to 100.000 EUR': 75000,
 '500.000 EUR or more': 500_000,
 'no bank deposits': np.nan}
dict_debt_midpoint = {
    '0 (no debt)': 0,
 '1 to 1.000 EUR': 500,
 '1.000 to 2.000 EUR': 1500,
 '10.000 to 20.000 EUR': 15_000,
 '100.000 EUR or more': 100_000,
 '2.000 to 5.000 EUR': 3_250,
 '20.000 to 40.000 EUR': 30_000,
 '40.000 to 60.000 EUR': 50_000,
 '5.000 to 10.000 EUR': 7_500,
 '60.000 to 100.000 EUR': 80_000,
 'no debt': 0}

In [106]:
df['wealth_bank'] = df['netwealth_detail_a'].map(dict_networth_mid_point)

In [107]:
df['debt'] = df['netwealth_detail_g'].map(dict_debt_midpoint)

In [108]:
df['is_homown'] = df['homeown'].apply(lambda x: x in ['live in own apartment',
                                                     'live in own house',
                                                     'own flat',
                                                     'own house',
                                                     'rent but own other home(s)'])

In [111]:
df = df.astype({k: v for k, v in dict_cols_parse.items() if k in df.columns})
df = df.replace({'nan': np.nan})

In [112]:
df.to_feather(os.path.join(DATA_DIR, 'hhp.feather'))